## Libraries Used

In [ ]:
# Data Handling
import pandas as pd
import numpy as np


# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.linear_model import LogisticRegression

# Model Evaluation
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score

# Display Formatting
from IPython.display import display, HTML

## Dataset extraction and organization
### Load and combine all datasets


In [ ]:
# Code here

## Data Cleaning and Preparation




In [ ]:
# Code here

## Exploratory Data Analysis
### Visualizing and analyzing sentiment distribution.

In [ ]:
# Code here

## Feature Engineering